In [18]:
using Rocket
using DataStructures

In [19]:
scheduler = PostponeScheduler()

source = of(1) |> schedule_on(scheduler)

subscription = subscribe!(source, logger())

Rocket.PostponedSubscription(PostponeScheduler(), Rocket.PostponedSubscriptionProps(false, false, VoidTeardown()))

In [21]:
release!(scheduler)

[LogActor] Data: 1
[LogActor] Completed


In [17]:
wait(scheduler)

[LogActor] Data: 1
[LogActor] Completed


In [18]:
q2 = Queue{Any}(q.store)

Queue{Any}(Deque [Any[]])

In [32]:
dump(q.store)

Deque{Any}
  nblocks: Int64 1
  blksize: Int64 1024
  len: Int64 0
  head: DataStructures.DequeBlock{Any}
    data: Array{Any}((1024,))
      1: Int64 1
      2: Int64 2
      3: #undef
      4: #undef
      5: #undef
      ...
      1020: #undef
      1021: #undef
      1022: #undef
      1023: #undef
      1024: #undef
    capa: Int64 1024
    front: Int64 1
    back: Int64 0
    prev: DataStructures.DequeBlock{Any}#= circular reference @-1 =#
    next: DataStructures.DequeBlock{Any}#= circular reference @-1 =#
  rear: DataStructures.DequeBlock{Any}
    data: Array{Any}((1024,))
      1: Int64 1
      2: Int64 2
      3: #undef
      4: #undef
      5: #undef
      ...
      1020: #undef
      1021: #undef
      1022: #undef
      1023: #undef
      1024: #undef
    capa: Int64 1024
    front: Int64 1
    back: Int64 0
    prev: DataStructures.DequeBlock{Any}#= circular reference @-1 =#
    next: DataStructures.DequeBlock{Any}#= circular reference @-1 =#
